# Notebook for Evaluation Upon AI Dialogues

## Table of Contents

- **Read config file**: read the configuration file that defines the experiment method, group by method, blacklist filter, etc.
- **Read dialogue messages**: read the dialogue file (in CSV format) containing the conversations to be analyzed.
- **Start analysis**: Initiate the analysis process using the configured experiment and grouping methods.
- **Deliver results and visualization**: visualize the obtained results in a suitable format for easy interpretation.


In [ ]:
from datetime import datetime
import os
import sys
import warnings
from dotenv import find_dotenv, load_dotenv
from IPython.display import HTML, display
from loguru import logger
import numpy as np
from openpyxl.styles import Font, PatternFill
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import yaml

load_dotenv(find_dotenv("../.env"))
warnings.filterwarnings("ignore")
logger.add("../log/runtime.log", format="{time:YYYY-MM-DD} | {level} | {message}", level="DEBUG", rotation="12:00",
           retention="7 days")
preset = os.environ.get("PRESET")

with open(os.path.join("../conf/preset", preset), "r") as ymlf:
    config = yaml.safe_load(ymlf)

In [ ]:
def allocate_lines(path: str) -> list:
    with open(os.path.join("../conf", path)) as reader:
        lines = [o.strip() for o in reader.readlines() if o.strip()]
    return lines

In [ ]:
def allocate_date(date_str: str) -> datetime:
    return datetime.strptime(date_str, "%Y-%m-%d")

In [ ]:
def export_xlsx(path: str, dataframe: pd.DataFrame) -> None:
    timestamp = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    with pd.ExcelWriter(os.path.join("../result", f"{timestamp} {path}"), engine="openpyxl", mode="w") as writer:
        dataframe.to_excel(writer, sheet_name="Statistics", engine="openpyxl")
        workbook = writer.book
        worksheet = writer.sheets["Statistics"]
        worksheet.column_dimensions["A"].width = 50

In [ ]:
source, mode, depth, channel, lang, weight, normalize, resample = config["SOURCE"], config["MODE"], config["DEPTH"], \
    config["CHANNEL"], config["LANG"], config["WEIGHT"], config["NORMALIZE"], config["RESAMPLE"]

In [ ]:
if mode in ["control", "full"]:
    before_start, before_end, after_start, after_end = map(allocate_date, [config["VERSIONS"]["control"]["start"],
                                                                           config["VERSIONS"]["control"]["end"],
                                                                           config["VERSIONS"]["experiment"]["start"],
                                                                           config["VERSIONS"]["experiment"]["end"]])
elif mode == "pop":
    before_start, before_end, after_start, after_end = map(allocate_date, [config["PERIODS"]["before"]["start"],
                                                                           config["PERIODS"]["before"][
                                                                               "end"],
                                                                           config["PERIODS"]["after"]["start"],
                                                                           config["PERIODS"]["after"]["end"]])
else:
    before_start, before_end, after_start, after_end = map(allocate_date, [config["VERSIONS"]["control"]["start"],
                                                                           config["VERSIONS"]["control"]["end"],
                                                                           config["VERSIONS"]["experiment"]["start"],
                                                                           config["VERSIONS"]["experiment"]["end"]])
    action = input("Press \"Enter\" to confirm using A/B control test date range, and press other keys to stop.")
    if action:
        sys.exit(1)

In [ ]:
expt_scope, ctrl_scope, bl_uid, bl_qname, bl_qid = map(
    allocate_lines, [
        config["VERSIONS"]["experiment"]["scope"],
        config["VERSIONS"]["control"]["scope"],
        config["BLACKLIST"]["uid"],
        config["BLACKLIST"]["qname"],
        config["BLACKLIST"]["qid"]])

In [ ]:
frame_lst = []
cols = ["chatdate", "biztype", "session_id", "session_type", "uid", "resp_time", "input_type", "relationguid", "qname",
        "expressid", "botexpid", "intentid", "intentname", "tomanualreason", "lang"]
for s in source:
    if s.endswith(".csv"):
        f = pd.read_csv(os.path.join("../src", s), sep=",", index_col=False, encoding="utf-8", dtype="unicode",
                        usecols=cols)
        frame_lst.append(f)
    elif s.endswith(".xlsx"):
        f = pd.read_excel(os.path.join("../src", s), index_col=False, engine="openpyxl", dtype="unicode", usecols=cols)
        frame_lst.append(f)
frame = pd.concat(frame_lst, ignore_index=True)
frame.shape

In [ ]:
frame["chatdate"] = pd.to_datetime(frame["chatdate"])
frame = frame[frame["lang"].isin(lang)]
frame = frame[frame["biztype"].isin(channel)]
frame = frame[(~frame["uid"].isin(bl_uid)) & (~frame["qname"].isin(bl_qname)) & (~frame["relationguid"].isin(bl_qid))]
frame = frame[(frame["chatdate"] >= before_start) & (frame["chatdate"] < after_end)]
frame["resp_time"] = pd.to_datetime(frame["resp_time"])
frame["chatweek"] = frame["chatdate"].dt.isocalendar().week
frame["chatmonth"] = frame["chatdate"].dt.month
frame.sort_values(by=["session_id", "resp_time"], ascending=True, inplace=True)
frame["session_type"] = frame["session_type"].replace(
    {"机器": "Chatbot", "人工": "Agent", "供应商客服": "Supplier Agent"})
frame["expressid"] = frame["relationguid"] + "-" + frame["expressid"]
frame["botexpid"] = frame["intentid"] + "-" + frame["botexpid"]
frame.shape

In [ ]:
if weight == "weighted":
    rounds = frame.groupby("session_id").size().reset_index(name="round")
    frame = frame.merge(rounds, on="session_id")
    if depth == "session":
        frame.rename(columns={"round": "weight"}, inplace=True)
    elif depth == "message":
        frame["weight"] = (1 / frame["round"]).round(decimals=2)
        frame.drop("round", axis=1, inplace=True)
        match normalize:
            case "minmax":
                scaler = MinMaxScaler()
                frame["weight"] = scaler.fit_transform(frame[["weight"]])
            case "log":
                frame["weight"] = np.log(frame["weight"])
frame.shape

In [ ]:
if mode in ["control", "full"]:
    expt_by = config["VERSIONS"]["experiment"]["by"]
    beta_frame = frame[(frame["chatdate"] >= before_start) & (frame["chatdate"] < before_end) & (
        frame[expt_by].isin(expt_scope))].dropna(subset=expt_by)
    if depth == "session" or weight in ["last", "weighted"]:
        beta_frame.drop_duplicates(subset="session_id", keep="last", inplace=True)
    beta_sessions = beta_frame["session_id"].tolist()
    beta_ssr = pd.DataFrame()
    if weight in ["equal", "last"]:
        beta_ssr = beta_frame.groupby([f"chat{resample}", "session_type"])["session_id"].count().unstack(fill_value=0)
        beta_ssr["Total"] = beta_ssr["Agent"] + beta_ssr["Chatbot"]
        beta_ssr["Total Diff"] = (100 * beta_ssr["Total"].diff() / beta_ssr["Total"]).round(decimals=2)
        beta_ssr["Prop"] = (100 * beta_ssr["Total"] / beta_frame.shape[0]).round(decimals=2)
        beta_ssr["SSR"] = (100 * beta_ssr["Chatbot"] / beta_ssr["Total"]).round(decimals=2)
        beta_ssr["SSR Diff"] = (100 * beta_ssr["SSR"].diff() / beta_ssr["SSR"]).round(decimals=2)
    elif weight == "weighted":
        beta_ssr = beta_frame.groupby([f"chat{resample}", "session_type"])["weight"].sum().unstack(fill_value=0)
        beta_ssr["Total"] = beta_ssr["Agent"] + beta_ssr["Chatbot"]
        beta_ssr["Total Diff"] = (100 * beta_ssr["Total"].diff() / beta_ssr["Total"]).round(decimals=2)
        beta_ssr["SSR"] = (100 * beta_ssr["Chatbot"] / beta_ssr["Total"]).round(decimals=2)
        beta_ssr["SSR Diff"] = (100 * beta_ssr["SSR"].diff() / beta_ssr["SSR"]).round(decimals=2)
    display(HTML(beta_ssr.to_html(notebook=True, max_rows=5)))
    export_xlsx("Beta Version Performance.xlsx", beta_ssr)

    ctrl_by = config["VERSIONS"]["control"]["by"]
    alpha_frame = frame[
        (frame["chatdate"] >= after_start) & (frame["chatdate"] < after_end) & (frame[ctrl_by].isin(ctrl_scope)) & (
            ~frame["session_id"].isin(beta_sessions))].dropna(
        subset=ctrl_by)
    alpha_ssr = pd.DataFrame()
    if depth == "session" or weight in ["last", "weighted"]:
        alpha_frame.drop_duplicates(subset="session_id", keep="last", inplace=True)
    if weight in ["equal", "last"]:
        alpha_ssr = alpha_frame.groupby([f"chat{resample}", "session_type"])["session_id"].count().unstack(fill_value=0)
        alpha_ssr["Total"] = alpha_ssr["Agent"] + alpha_ssr["Chatbot"]
        alpha_ssr["Total Diff"] = (100 * alpha_ssr["Total"].diff() / alpha_ssr["Total"]).round(decimals=2)
        alpha_ssr["Prop"] = (100 * alpha_ssr["Total"] / alpha_frame.shape[0]).round(decimals=2)
        alpha_ssr["SSR"] = (100 * alpha_ssr["Chatbot"] / alpha_ssr["Total"]).round(decimals=2)
        alpha_ssr["SSR Diff"] = (100 * alpha_ssr["SSR"].diff() / alpha_ssr["SSR"]).round(decimals=2)
    elif weight == "weighted":
        alpha_ssr = alpha_frame.groupby([f"chat{resample}", "session_type"])["weight"].sum().unstack(fill_value=0)
        alpha_ssr["Total"] = alpha_ssr["Agent"] + alpha_ssr["Chatbot"]
        alpha_ssr["Total Diff"] = (100 * alpha_ssr["Total"].diff() / alpha_ssr["Total"]).round(decimals=2)
        alpha_ssr["SSR"] = (100 * alpha_ssr["Chatbot"] / alpha_ssr["Total"]).round(decimals=2)
        alpha_ssr["SSR Diff"] = (100 * alpha_ssr["SSR"].diff() / alpha_ssr["SSR"]).round(decimals=2)
    display(HTML(alpha_ssr.to_html(notebook=True, max_rows=5)))
    export_xlsx("Alpha Version Performance.xlsx", alpha_ssr)
    
    # TODO: plot line chart and heatmap

In [ ]:
if mode in ["pop", "full"]:
    pop_by = config["PERIODS"]["by"]

    before_frame = frame[(frame["chatdate"] >= before_start) & (frame["chatdate"] < before_end)].dropna(subset=pop_by)
    before_ssr = pd.DataFrame()
    if depth == "session" or weight in ["last", "weighted"]:
        before_frame.drop_duplicates(subset="session_id", keep="last", inplace=True)
    if weight in ["equal", "last"]:
        before_ssr = before_frame.groupby([pop_by, "session_type"])["session_id"].count().unstack(fill_value=0)
        before_ssr["Total"] = before_ssr["Agent"] + before_ssr["Chatbot"]
        before_ssr["Prop"] = (100 * before_ssr["Total"] / before_frame.shape[0]).round(decimals=2)
        before_ssr["SSR"] = (100 * before_ssr["Chatbot"] / before_ssr["Total"]).round(decimals=2)
        before_ssr.sort_values(by=["Chatbot", "Prop", "SSR"], ascending=False, inplace=True)
    elif weight == "weighted":
        before_ssr = before_frame.groupby([pop_by, "session_type"])["weight"].sum().unstack(fill_value=0)
        before_ssr["Total"] = before_ssr["Agent"] + before_ssr["Chatbot"]
        before_ssr["SSR"] = (100 * before_ssr["Chatbot"] / before_ssr["Total"]).round(decimals=2)

    after_frame = frame[(frame["chatdate"] >= after_start) & (frame["chatdate"] < after_end)].dropna(subset=pop_by)
    after_ssr = pd.DataFrame()
    if depth == "session" or weight in ["last", "weighted"]:
        after_frame.drop_duplicates(subset="session_id", keep="last", inplace=True)
    if weight in ["equal", "last"]:
        after_ssr = after_frame.groupby([pop_by, "session_type"])["session_id"].count().unstack(fill_value=0)
        after_ssr["Total"] = after_ssr["Agent"] + after_ssr["Chatbot"]
        after_ssr["Prop"] = (100 * after_ssr["Total"] / after_frame.shape[0]).round(decimals=2)
        after_ssr["SSR"] = (100 * after_ssr["Chatbot"] / after_ssr["Total"]).round(decimals=2)
        after_ssr.sort_values(by=["Chatbot", "Prop", "SSR"], ascending=False, inplace=True)
    elif weight == "weighted":
        after_ssr = after_frame.groupby([pop_by, "session_type"])["weight"].sum().unstack(fill_value=0)
        after_ssr["Total"] = after_ssr["Agent"] + after_ssr["Chatbot"]
        after_ssr["SSR"] = (100 * after_ssr["Chatbot"] / after_ssr["Total"]).round(decimals=2)

    merge_ssr = before_ssr.merge(after_ssr, on=pop_by, how="outer", suffixes=("(before)", "(after)")).sort_values(
        by=["Agent(after)", "SSR(after)"], ascending=False)
    merge_ssr["Total Fluctuation"] = (
            100 * (merge_ssr["Total(after)"] - merge_ssr["Total(before)"]) / merge_ssr["Total(before)"]).round(2)
    merge_ssr["SSR Fluctuation"] = (
            100 * (merge_ssr["SSR(after)"] - merge_ssr["SSR(before)"]) / merge_ssr["SSR(before)"]).round(2)
    display(HTML(merge_ssr.to_html(notebook=True, max_rows=5)))
    export_xlsx("Merged Period-on-Period Performance.xlsx", merge_ssr)
    
    # TODO: plot line chart and heatmap